In [4]:
import csv
import numpy as np
from collections import Counter
import random

In [5]:
file = open("C:\\Users\\Simeon Horner\\Documents\\GitHub\\RiskAndLogisticsA2\\Data Files\\CSV versions\\OrderList.csv", encoding="utf-8")
csvreader = csv.reader(file)

Orders = []

for row in csvreader:
    Orders.append([int(x) for x in row[1:]])

Distances = np.genfromtxt('C:\\Users\\Simeon Horner\\Documents\\GitHub\\RiskAndLogisticsA2\\Data Files\\CSV versions\\DistanceMatrix_meters.csv', delimiter=',', skip_header=0, dtype='i')
Distances[0, 0] = 0

In [13]:
def ConstructionHeuristic_random():
    newShelves = [0]*96
    currShelf = 0

    tempProducts = [x for x in range(1, 91)]

    for i in range(len(newShelves)):
        random_prod = random.choice(tempProducts)
        tempProducts.remove(random_prod)
        newShelves[currShelf] = random_prod
        currShelf += 1

        if len(tempProducts) == 0:
            tempProducts = [x for x in range(1, 91)]

    return newShelves




In [14]:
Shelves = ConstructionHeuristic_random()

print(Shelves)

[59, 75, 57, 2, 62, 66, 43, 50, 18, 76, 1, 23, 38, 17, 78, 56, 40, 48, 69, 80, 86, 32, 34, 10, 89, 70, 7, 54, 68, 88, 13, 77, 30, 28, 27, 20, 45, 61, 12, 71, 21, 87, 22, 5, 82, 31, 16, 60, 36, 65, 26, 74, 44, 58, 35, 6, 83, 9, 79, 72, 42, 85, 39, 3, 53, 73, 11, 25, 81, 15, 37, 14, 33, 64, 67, 29, 24, 46, 47, 41, 90, 8, 84, 19, 4, 52, 49, 51, 63, 55, 8, 73, 1, 3, 68, 85]
